# **Wan2.2 Animate for character animation and replacement in ComfyUI (WIP)**

---


- Run the cell below to get a link (e.g. https://localhost:8188/) which you can use to launch the comfyUI interface. Remember to switch models in the workflows to either safetensors or GGUF depending on the model format you selected for download.
- You can get the workflows here: https://github.com/Isi-dev/Google-Colab_Notebooks/tree/main/ComfyUI/Wan_2_2_Animate
- Github project page: https://github.com/Wan-Video/Wan2.2
- Notebook source: https://github.com/Isi-dev/Google-Colab_Notebooks
- Premium notebooks I highly recommend: https://isinse.gumroad.com/
- Google Colab Youtube Playlist: https://www.youtube.com/playlist?list=PLdi1sS5pbSYeA470Sb1wARR4OieCBIqMv
- Even $1 helps support my work: https://buymeacoffee.com/isiomo



In [1]:
# @markdown # 💥Prepare Environment


#Animate Q_8 GGUF: https://huggingface.co/Kijai/WanVideo_comfy_GGUF/resolve/main/Wan22Animate/Wan2_2_Animate_14B_Q8_0.gguf

#Animate Q4_k_M GGUF: https://huggingface.co/Kijai/WanVideo_comfy_GGUF/resolve/main/Wan22Animate/Wan2_2_Animate_14B_Q4_K_M.gguf






!pip install torch==2.8.0 torchvision==0.23.0

# using_T4_GPU = True # @param {type:"boolean"}
using_T4_GPU = False
include_manager = True # @param {type:"boolean"}

%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate xformers==0.0.32.post1 triton==3.4 sageattention
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!pip install color-matcher
!pip install onnxruntime-gpu -y
!git clone https://github.com/comfyanonymous/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()


%cd /content/ComfyUI/custom_nodes
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
if include_manager:
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
!git clone --branch forQwen https://github.com/Isi-dev/ComfyUI_GGUF.git
!git clone https://github.com/Isi-dev/ComfyUI_DeleteModelPassthrough.git
!git clone https://github.com/Isi-dev/comfyui_controlnet_aux
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/kijai/ComfyUI-segment-anything-2
!git clone https://github.com/kijai/ComfyUI-Florence2
!git clone https://github.com/john-mnz/ComfyUI-Inspyrenet-Rembg.git
!git clone https://github.com/Isi-dev/ComfyUI_Animation_Nodes_and_Workflows
%cd /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_DeleteModelPassthrough
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/comfyui_controlnet_aux
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-WanVideoWrapper
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-VideoHelperSuite
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-KJNodes
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-Florence2
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-Inspyrenet-Rembg
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI_Animation_Nodes_and_Workflows
!pip install -r requirements.txt
if include_manager:
    %cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
    !pip install -r requirements.txt

clear_output()


%cd /content/ComfyUI


import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import subprocess
import sys
from pathlib import Path
# sys.path.insert(0, '/content/ComfyUI')

def install_apt_packages():
    packages = ['aria2']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")


print("Installing apt packages...")
install_apt_packages()

def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1]
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename



def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename

def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False


model_GGUF_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_GGUF/resolve/main/Wan22Animate/Wan2_2_Animate_14B_Q4_K_M.gguf"# @param {"type":"string"}
# model_lowNoiseGGUF_download_url = "https://huggingface.co/bullerwins/Wan2.2-T2V-A14B-GGUF/resolve/main/wan2.2_t2v_low_noise_14B_Q4_K_M.gguf"# @param {"type":"string"}
# model_I2V_highNoiseGGUF_download_url = "https://huggingface.co/bullerwins/Wan2.2-I2V-A14B-GGUF/resolve/main/wan2.2_i2v_high_noise_14B_Q4_K_M.gguf"# @param {"type":"string"}
# model_I2V_lowNoiseGGUF_download_url = "https://huggingface.co/bullerwins/Wan2.2-I2V-A14B-GGUF/resolve/main/wan2.2_i2v_low_noise_14B_Q4_K_M.gguf"# @param {"type":"string"}
# vaceBlocks_highNoiseGGUF_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_GGUF/resolve/main/VACE/Wan2_2_Fun_VACE_module_A14B_HIGH_Q4_K_M.gguf"# @param {"type":"string"}
# vaceBlocks_lowNoiseGGUF_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_GGUF/resolve/main/VACE/Wan2_2_Fun_VACE_module_A14B_LOW_Q4_K_M.gguf"# @param {"type":"string"}


download_safetensors_instead = True # @param {type:"boolean"}
model_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/Wan22Animate/Wan2_2-Animate-14B_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}
# model_lowNoise_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/T2V/Wan2_2-T2V-A14B-LOW_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}
# model_I2V_highNoise_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/I2V/Wan2_2-I2V-A14B-HIGH_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}
# model_I2V_lowNoise_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/I2V/Wan2_2-I2V-A14B-LOW_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}
# vaceBlocks_highNoise_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/VACE/Wan2_2_Fun_VACE_module_A14B_HIGH_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}
# vaceBlocks_lowNoise_download_url = "https://huggingface.co/Kijai/WanVideo_comfy_fp8_scaled/resolve/main/VACE/Wan2_2_Fun_VACE_module_A14B_LOW_fp8_e4m3fn_scaled_KJ.safetensors"# @param {"type":"string"}



if download_safetensors_instead:
    model_download(model_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_lowNoise_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_I2V_highNoise_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_I2V_lowNoise_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(vaceBlocks_highNoise_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(vaceBlocks_lowNoise_download_url, "/content/ComfyUI/models/diffusion_models")
else:
    model_download(model_GGUF_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_lowNoiseGGUF_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_I2V_highNoiseGGUF_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(model_I2V_lowNoiseGGUF_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(vaceBlocks_highNoiseGGUF_download_url, "/content/ComfyUI/models/diffusion_models")
    # model_download(vaceBlocks_lowNoiseGGUF_download_url, "/content/ComfyUI/models/diffusion_models")


text_encoder = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors"# @param {"type":"string"}
vae = "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors"# @param {"type":"string"}
clip_vision = "https://huggingface.co/Isi99999/Wan_Extras/resolve/main/clip_vision_h.safetensors"# @param {"type":"string"}


model_download(text_encoder, "/content/ComfyUI/models/text_encoders")
model_download(vae, "/content/ComfyUI/models/vae")
model_download(clip_vision, "/content/ComfyUI/models/clip_vision")


download_high_noise_speed_LoRA = True # @param {type:"boolean"}
high_noise_speed_LoRA_download_url = "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank32_bf16.safetensors"# @param {"type":"string"}
download_low_noise_speed_LoRA = True # @param {type:"boolean"}
low_noise_speed_LoRA_download_url = "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Lightx2v/lightx2v_T2V_14B_cfg_step_distill_v2_lora_rank32_bf16.safetensors"# @param {"type":"string"}

if download_high_noise_speed_LoRA:
    lightx2v_lora = model_download(high_noise_speed_LoRA_download_url, "/content/ComfyUI/models/loras")
if download_low_noise_speed_LoRA:
    lightx2v_lora_lowNoise = model_download(low_noise_speed_LoRA_download_url, "/content/ComfyUI/models/loras")

download_segmentation_model = True # @param {type:"boolean"}
segmentation_model = "https://huggingface.co/Kijai/sam2-safetensors/resolve/main/sam2.1_hiera_small.safetensors"# @param {"type":"string"}
if download_segmentation_model:
    model_download(segmentation_model, "/content/ComfyUI/models/sam2")


download_loRA1 = True # @param {type:"boolean"}
lora1_download_url = "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_animate_14B_relight_lora_bf16.safetensors"# @param {"type":"string"}

download_loRA_2 = False # @param {type:"boolean"}
lora_2_download_url = "Put your loRA here"# @param {"type":"string"}

download_loRA_3 = False # @param {type:"boolean"}
lora3_download_url = "Put your loRA here"# @param {"type":"string"}

download_loRA_4 = False # @param {type:"boolean"}
lora_4_download_url = "Put your loRA here"# @param {"type":"string"}

token_if_civitai_url = ""# @param {"type":"string"}



lora1 = None
if download_loRA1:
    lora1 = download_lora(lora1_download_url, civitai_token=token_if_civitai_url)
# Validate loRA file extension
valid_extensions = {'.safetensors', '.ckpt', '.pt', '.pth', '.sft'}
if lora1:
    if not any(lora1.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora1}")
        lora1 = None
    else:
        clear_output()
        print("loRA1 downloaded succesfully!")

lora_2 = None
if download_loRA_2:
    lora_2 = download_lora(lora_2_download_url, civitai_token=token_if_civitai_url)
if lora_2:
    if not any(lora_2.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_2}")
        lora_2 = None
    else:
        clear_output()
        print("loRA 2 downloaded succesfully!")

lora_3 = None
if download_loRA_3:
    lora_3 = download_lora(lora3_download_url, civitai_token=token_if_civitai_url)
if lora_3:
    if not any(lora_3.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_3}")
        lora_3 = None
    else:
        clear_output()
        print("loRA 3 downloaded succesfully!")

lora_4 = None
if download_loRA_4:
    lora_4 = download_lora(lora_4_download_url, civitai_token=token_if_civitai_url)
if lora_4:
    if not any(lora_4.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_4}")
        lora_4 = None
    else:
        clear_output()
        print("loRA 4 downloaded succesfully!")


# model = model_download(flux_model_download_url, "/content/ComfyUI/models/unet")
# text_encoder = model_download(flux_text_encoder_download_url, "/content/ComfyUI/models/clip")
# text_encoder2 = model_download(flux_text_encoder_download_url2, "/content/ComfyUI/models/clip")
# vae = model_download(flux_vae_download_url, "/content/ComfyUI/models/vae")
# lora = model_download(flux_uso_lora_download_url, "/content/ComfyUI/models/loras")
# patch = model_download(flux_uso_patch_download_url, "/content/ComfyUI/models/model_patches")
# clip_vision = model_download(clip_vision_download_url, "/content/ComfyUI/models/clip_vision")
# turbo = model_download(speed_lora_download_url, "/content/ComfyUI/models/loras")

use_cloudflare = False # @param {type:"boolean"}
use_interface_in_cell = False # @param {type:"boolean"}


if use_cloudflare:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb

    import subprocess
    import threading
    import time
    import socket
    import urllib.request

    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

      p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
          print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
        #print(l, end='')
    clear_output()

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server


elif use_interface_in_cell:
    import threading
    import time
    import socket
    def iframe_thread(port):
      while True:
          time.sleep(0.5)
          sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
          result = sock.connect_ex(('127.0.0.1', port))
          if result == 0:
            break
          sock.close()
      from google.colab import output
      output.serve_kernel_port_as_iframe(port, height=1024)
      clear_output()
      print("to open it in a window you can open this link here:")
      output.serve_kernel_port_as_window(port)

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

else:
    import socket, time, threading
    from google.colab import output

    def link_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        clear_output()
        print("Click the link below to launch the comfyui interface")
        output.serve_kernel_port_as_window(port)


    # Start thread for port 8188
    threading.Thread(target=link_thread, daemon=True, args=(8188,)).start()

    if using_T4_GPU:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server








Click the link below to launch the comfyui interface
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

FETCH ComfyRegistry Data: 5/98
FETCH ComfyRegistry Data: 10/98
FETCH ComfyRegistry Data: 15/98
FETCH ComfyRegistry Data: 20/98
FETCH ComfyRegistry Data: 25/98
FETCH ComfyRegistry Data: 30/98
FETCH ComfyRegistry Data: 35/98
FETCH ComfyRegistry Data: 40/98
FETCH ComfyRegistry Data: 45/98
FETCH ComfyRegistry Data: 50/98
FETCH ComfyRegistry Data: 55/98
FETCH ComfyRegistry Data: 60/98
FETCH ComfyRegistry Data: 65/98
FETCH ComfyRegistry Data: 70/98
FETCH ComfyRegistry Data: 75/98
FETCH ComfyRegistry Data: 80/98
FETCH ComfyRegistry Data: 85/98
FETCH ComfyRegistry Data: 90/98
FETCH ComfyRegistry Data: 95/98
FETCH ComfyRegistry Data [DONE]
[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]
[ComfyUI-Manager] All startup tasks have been completed.
got prompt
Failed to find /content/ComfyUI/custom_nodes/comfyui_controlnet_aux/ckpts/hr16/yolox-onnx/yolox_l.torchscript.pt.
